# Notebook to highlight SK_ID_CURR that need to be tested to improve app robustness

When transitionning from requests using large batches of data to single id at the time a lot of issues show up and we need to establish a list of ids to highlight these potential issues and solve them to make the api and the streamlit app more robust

For example : A client who only had short term loans in the past when we use their id to do a request to the api at some point in the prepare input data function we expect both long term and short term columns after doing the one hot encoding with get dummies however this then leads to an error given that we are missing a column

## Importation librairies

In [1]:
import pandas as pd
import numpy as np

## Importation des datasets

In [2]:
HomeCredit_ini = pd.read_csv('data/HomeCredit_columns_description.csv', encoding='latin-1')
POS_CASH_balance_ini = pd.read_csv('data/POS_CASH_balance.csv')
installments_payments_ini = pd.read_csv('data/installments_payments.csv')
previous_application_ini = pd.read_csv('data/previous_application.csv')
application_test_ini = pd.read_csv('data/application_test.csv')
application_train_ini = pd.read_csv('data/application_train.csv')
bureau_balance_ini = pd.read_csv('data/bureau_balance.csv')
bureau_ini = pd.read_csv('data/bureau.csv')
sample_submission_ini = pd.read_csv('data/sample_submission.csv')
credit_card_balance_ini = pd.read_csv('data/credit_card_balance.csv')
features_manual_and_func_from_first_three_with_app_train_ini = pd.read_csv('data/features_manual_and_func_from_first_three_with_app_train.csv')

In [39]:
HomeCredit = HomeCredit_ini.copy()
POS_CASH_balance = POS_CASH_balance_ini.copy()
installments_payments = installments_payments_ini.copy()
previous_application = previous_application_ini.copy()
application_test = application_test_ini.copy()
application_train = application_train_ini.copy()
bureau_balance = bureau_balance_ini.copy()
bureau = bureau_ini.copy()
sample_submission = sample_submission_ini.copy()
credit_card_balance = credit_card_balance_ini.copy()
features_manual_and_func_from_first_three_with_app_train = features_manual_and_func_from_first_three_with_app_train_ini.copy()

In [4]:
credit_card_balance.columns

Index(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'AMT_BALANCE',
       'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT',
       'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT',
       'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY',
       'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT',
       'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE',
       'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT',
       'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT',
       'CNT_INSTALMENT_MATURE_CUM', 'NAME_CONTRACT_STATUS', 'SK_DPD',
       'SK_DPD_DEF'],
      dtype='object')

# Ids table

In [40]:
# --- Code to generate the ID presence DataFrame ---

# 1. Get all unique SK_ID_CURR from the main application table (e.g., application_test)
# We'll use these IDs as the rows in our output DataFrame.
master_ids = sorted(list(application_test['SK_ID_CURR'].unique()))

# 2. For tables directly linked by SK_ID_CURR, get sets of their present SK_ID_CURR for efficient lookup
bureau_present_curr_ids = set(bureau['SK_ID_CURR'].unique())
prev_app_present_curr_ids = set(previous_application['SK_ID_CURR'].unique())

# 3. For tables linked via SK_ID_BUREAU (from bureau.csv):
#    a. Create a mapping from SK_ID_CURR to a set of its SK_ID_BUREAU(s)
#       A single SK_ID_CURR can have multiple SK_ID_BUREAU entries.
curr_to_bureau_map = bureau.groupby('SK_ID_CURR')['SK_ID_BUREAU'].apply(set).to_dict()
#    b. Get a set of all SK_ID_BUREAU present in bureau_balance.csv
bureau_balance_present_bureau_ids = set(bureau_balance['SK_ID_BUREAU'].unique())

# 4. For tables linked via SK_ID_PREV (from previous_application.csv):
#    a. Create a mapping from SK_ID_CURR to a set of its SK_ID_PREV(s)
#       A single SK_ID_CURR can have multiple SK_ID_PREV entries.
curr_to_prev_map = previous_application.groupby('SK_ID_CURR')['SK_ID_PREV'].apply(set).to_dict()
#    b. Get sets of all SK_ID_PREV present in their respective tables
pos_cash_present_prev_ids = set(POS_CASH_balance['SK_ID_PREV'].unique())
installments_present_prev_ids = set(installments_payments['SK_ID_PREV'].unique())
credit_card_present_prev_ids = set(credit_card_balance['SK_ID_PREV'].unique())


# 5. Build the result DataFrame
results_data = []

for sk_id_curr in master_ids:
    row = {'SK_ID_CURR': sk_id_curr}

    # Check presence in application_test (always True by definition here)
    row['in_application_test'] = True # Or check against application_test['SK_ID_CURR'] if master_ids came from elsewhere

    # Check direct links
    row['in_bureau'] = sk_id_curr in bureau_present_curr_ids
    row['in_previous_application'] = sk_id_curr in prev_app_present_curr_ids

    # Check bureau_balance (via bureau.csv)
    # Does this sk_id_curr have any related bureau_ids that are in bureau_balance_present_bureau_ids?
    related_bureau_ids = curr_to_bureau_map.get(sk_id_curr, set()) # Get set of SK_ID_BUREAU for this SK_ID_CURR
    row['in_bureau_balance'] = any(b_id in bureau_balance_present_bureau_ids for b_id in related_bureau_ids)

    # Check tables linked by SK_ID_PREV (via previous_application.csv)
    related_prev_ids = curr_to_prev_map.get(sk_id_curr, set()) # Get set of SK_ID_PREV for this SK_ID_CURR
    row['in_POS_CASH_balance'] = any(p_id in pos_cash_present_prev_ids for p_id in related_prev_ids)
    row['in_installments_payments'] = any(p_id in installments_present_prev_ids for p_id in related_prev_ids)
    row['in_credit_card_balance'] = any(p_id in credit_card_present_prev_ids for p_id in related_prev_ids)

    results_data.append(row)

id_presence_df = pd.DataFrame(results_data)

# Set SK_ID_CURR as index if desired
# id_presence_df = id_presence_df.set_index('SK_ID_CURR')

In [41]:
id_presence_df

,SK_ID_CURR,in_application_test,in_bureau,in_previous_application,in_bureau_balance,in_POS_CASH_balance,in_installments_payments,in_credit_card_balance
0,100001,True,True,True,True,True,True,False
1,100005,True,True,True,True,True,True,False
2,100013,True,True,True,True,True,True,False
3,100028,True,True,True,True,True,True,True
4,100038,True,False,True,False,True,True,False
...,...,...,...,...,...,...,...,...
48739,456221,True,True,True,True,True,True,False
48740,456222,True,False,True,False,True,True,False
48741,456223,True,True,True,True,True,True,False
48742,456224,True,True,True,True,True,True,False


In [44]:
id_presence_df[id_presence_df['in_application_test'] == True]

,SK_ID_CURR,in_application_test,in_bureau,in_previous_application,in_bureau_balance,in_POS_CASH_balance,in_installments_payments,in_credit_card_balance
0,100001,True,True,True,True,True,True,False
1,100005,True,True,True,True,True,True,False
2,100013,True,True,True,True,True,True,False
3,100028,True,True,True,True,True,True,True
4,100038,True,False,True,False,True,True,False
...,...,...,...,...,...,...,...,...
48739,456221,True,True,True,True,True,True,False
48740,456222,True,False,True,False,True,True,False
48741,456223,True,True,True,True,True,True,False
48742,456224,True,True,True,True,True,True,False


In [43]:
id_presence_df[id_presence_df['in_installments_payments'] == False]

,SK_ID_CURR,in_application_test,in_bureau,in_previous_application,in_bureau_balance,in_POS_CASH_balance,in_installments_payments,in_credit_card_balance
32,100241,True,True,False,True,False,False,False
33,100245,True,True,False,True,False,False,False
37,100278,True,True,True,True,False,False,False
51,100399,True,True,False,True,False,False,False
148,101107,True,True,False,True,False,False,False
...,...,...,...,...,...,...,...,...
48540,454578,True,False,False,False,False,False,False
48577,454841,True,True,True,True,False,False,False
48583,454908,True,True,False,True,False,False,False
48709,455958,True,True,False,True,False,False,False


In [52]:
id_presence_df[(id_presence_df['in_previous_application'] == False) & (id_presence_df['in_POS_CASH_balance'] == False) & (id_presence_df['in_installments_payments'] == False) & (id_presence_df['in_credit_card_balance'] == False) & (id_presence_df['in_bureau'] == False) & (id_presence_df['in_bureau_balance'] == False)]

,SK_ID_CURR,in_application_test,in_bureau,in_previous_application,in_bureau_balance,in_POS_CASH_balance,in_installments_payments,in_credit_card_balance
192,101332,True,False,False,False,False,False,False
238,101614,True,False,False,False,False,False,False
410,102878,True,False,False,False,False,False,False
625,104376,True,False,False,False,False,False,False
767,105292,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
47924,450180,True,False,False,False,False,False,False
48099,451383,True,False,False,False,False,False,False
48250,452495,True,False,False,False,False,False,False
48494,454245,True,False,False,False,False,False,False


## Dedicated to reading column descriptions

In [5]:
pd.set_option('display.max_colwidth', None)

In [6]:
HomeCredit[HomeCredit['Table'] == 'bureau_balance.csv']

,Unnamed: 0,Table,Row,Description,Special
139,142,bureau_balance.csv,SK_BUREAU_ID,Recoded ID of Credit Bureau credit (unique coding for each application) - use this to join to CREDIT_BUREAU table,hashed
140,143,bureau_balance.csv,MONTHS_BALANCE,Month of balance relative to application date (-1 means the freshest balance date),time only relative to the application
141,144,bureau_balance.csv,STATUS,"Status of Credit Bureau loan during the month (active, closed, DPD0-30, [C means closed, X means status unknown, 0 means no DPD, 1 means maximal did during month between 1-30, 2 means DPD 31-60, 5 means DPD 120+ or sold or written off ] )",NaN


# Troubleshooting loan types (in terms of duration) from bureau balance

Lets establish pick a list of ids which would certainly cause issues when inputed alone in prepare_input_data()

## Only long term loans for a certain SK_ID_BUREAU

In [7]:
bureau_balance[bureau_balance['MONTHS_BALANCE'] == -60]

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
186,5715455,-60,X
361,5715529,-60,C
606,5717356,-60,0
673,5717363,-60,C
736,5717364,-60,C
...,...,...,...
27299546,5025218,-60,0
27299591,5041084,-60,C
27299674,5041120,-60,C
27299762,5041143,-60,C


In [8]:
bureau_balance[bureau_balance['SK_ID_BUREAU'] == 5025218].tail(17)

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
27299536,5025218,-50,C
27299537,5025218,-51,C
27299538,5025218,-52,C
27299539,5025218,-53,C
27299540,5025218,-54,C
27299541,5025218,-55,0
27299542,5025218,-56,0
27299543,5025218,-57,0
27299544,5025218,-58,0
27299545,5025218,-59,0


This customer only has one long term loan

# Client for never applied for a loan in the past

In [9]:
bureau

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,0.0,Microloan,-19,NaN
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,Consumer credit,-2493,NaN
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,0.0,0.0,Consumer credit,-1508,NaN


In [10]:
set(list(application_test['SK_ID_CURR']))-set(list(bureau['SK_ID_CURR']))

{327689,
 196619,
 163854,
 196623,
 327702,
 393256,
 294952,
 294953,
 393269,
 196667,
 196669,
 262213,
 360518,
 163929,
 360539,
 393312,
 196745,
 196747,
 163981,
 393361,
 163988,
 426132,
 393370,
 131242,
 393395,
 196826,
 327908,
 327909,
 229610,
 295155,
 262392,
 262399,
 164095,
 229632,
 131333,
 164106,
 295183,
 327954,
 360723,
 196889,
 229659,
 229663,
 131366,
 229675,
 131373,
 262446,
 295216,
 164147,
 393543,
 295255,
 131426,
 393573,
 229738,
 360812,
 196977,
 229745,
 393587,
 360838,
 262535,
 262537,
 262538,
 328074,
 131479,
 393628,
 131498,
 229802,
 360882,
 295348,
 262581,
 360885,
 360893,
 426436,
 393671,
 328144,
 197075,
 328154,
 262620,
 295389,
 229858,
 262629,
 131565,
 393709,
 197103,
 360941,
 295421,
 393738,
 393740,
 426510,
 295443,
 197140,
 197152,
 229921,
 295505,
 361051,
 262752,
 229984,
 131688,
 131690,
 328303,
 230038,
 393886,
 164513,
 131759,
 197295,
 164542,
 426688,
 361152,
 262862,
 262865,
 393940,
 131798,
 

In [11]:
pd.DataFrame(set(list(application_test['SK_ID_CURR']))-set(list(bureau['SK_ID_CURR'])), columns=['IDs'])

,IDs
0,327689
1,196619
2,163854
3,196623
4,327702
...,...
6419,196587
6420,425965
6421,425968
6422,360439


En testant avec le 327689 étant donné qu'il ne présente meme pas d'id bureau étant donné que cet id n'est présent que dans application test on a l'erreur suivante : API Response (first 500 chars): {"error":"An unexpected error occurred: 'SK_ID_BUREAU'"} ...

In [12]:
bureau[bureau['SK_ID_BUREAU'] == 327689]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY


# Test

In [13]:
# --- Aggrégation de bureau et bureau_balance ---

if bureau_balance.shape == (0,0) :
    bureau_balance = pd.DataFrame(columns=initial_expected_columns_bureau_balance)

if bureau.shape == (0,0) :
    bureau= pd.DataFrame(columns=initial_expected_columns_bureau)

# Création des features "manuelles"
bureau_balance_loan_duration_months = bureau_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].count().reset_index()
bureau_balance_loan_duration_months.rename(columns = {'MONTHS_BALANCE' : 'MONTHS_LOAN_DURATION'}, inplace=True)

bureau_balance_last_known_loan_status = bureau_balance.sort_values(by='MONTHS_BALANCE', ascending=False).groupby('SK_ID_BUREAU').first().reset_index()[['SK_ID_BUREAU', 'STATUS']]
#bureau_balance_last_known_loan_status = pd.get_dummies(bureau_balance_last_known_loan_status, dtype=int)
all_expected_status_categories = ['0', '1', '2', '3', '4', '5', 'C', 'X'] # These are the raw STATUS values
for cat_col_name in all_expected_status_categories:
    if cat_col_name not in bureau_balance_last_known_loan_status.columns:
        bureau_balance_last_known_loan_status[cat_col_name] = 0

bureau_balance_for_dpd_flag = bureau_balance.copy()
non_dpd_statuses = ['C', 'X', '0']
dpd_flagged = ~bureau_balance_for_dpd_flag['STATUS'].isin(non_dpd_statuses) #Cree True/False value en fonction de la condition ici dpd --> True
bureau_balance_for_dpd_flag['DPD_FLAG'] = dpd_flagged.astype(int)
bureau_balance_nombre_delais_de_paiements = bureau_balance_for_dpd_flag.groupby(by='SK_ID_BUREAU')['DPD_FLAG'].sum().reset_index()

bureau_balance_nombre_de_delais_de_paiements_par_categorie = bureau_balance.groupby('SK_ID_BUREAU')['STATUS'].value_counts().unstack(fill_value=0).reset_index()
bureau_balance_nombre_de_delais_de_paiements_par_categorie.columns.name = None

Values_to_map = {'C':0, 'X':0, '0':0, '1':15, '2':45, '3':75, '4':105, '5':120}
bureau_balance_for_mean_delay_calc = bureau_balance.copy()
bureau_balance_for_mean_delay_calc['MEAN_DAYS_PAST_DUE'] = bureau_balance_for_mean_delay_calc['STATUS'].map(Values_to_map).fillna('Unknown')
bureau_balance_duree_moyenne_delais_de_paiements = bureau_balance_for_mean_delay_calc.groupby('SK_ID_BUREAU')['MEAN_DAYS_PAST_DUE'].mean().reset_index()

bureau_balance_loan_duration_categorised = bureau_balance.sort_values(by='MONTHS_BALANCE', ascending=True).groupby('SK_ID_BUREAU').first().reset_index()
bureau_balance_loan_duration_categorised['YEAR_LOAN_DURATION'] = round(bureau_balance_loan_duration_categorised['MONTHS_BALANCE']/(-12), 1)
bureau_balance_loan_duration_categorised['LOAN_TYPE'] = np.where(
    bureau_balance_loan_duration_categorised['YEAR_LOAN_DURATION'] >= 5, # Condition
    'Long Term',                                             # Value if True
    'Short Term'                                             # Value if False
)
bureau_balance_loan_duration_categorised = bureau_balance_loan_duration_categorised.drop(columns=['MONTHS_BALANCE', 'STATUS'], axis=0)
bureau_balance_loan_duration_categorised = pd.get_dummies(bureau_balance_loan_duration_categorised, dtype=int)

# Failproofnet loan type
expected_loan_type_dummies = ['LOAN_TYPE_Long Term', 'LOAN_TYPE_Short Term']
for col_name_to_ensure in expected_loan_type_dummies:
    if col_name_to_ensure not in bureau_balance_loan_duration_categorised.columns:
    # If the DataFrame is not empty, assign 0. 
    # If it's empty, assign an empty Series of the correct dtype.
    # This ensures the column exists for schema consistency in merges.
        if not bureau_balance_loan_duration_categorised.empty:
            bureau_balance_loan_duration_categorised[col_name_to_ensure] = 0
        else:
        # If bureau_balance_loan_duration_categorised is truly empty (no rows, no SK_ID_BUREAU),
        # creating just an empty Series might still cause issues if SK_ID_BUREAU is needed
        # for the merge into final_bureau_balance_features.
        # However, the goal here is to prevent the KeyError when these columns are *selected*.
        # If this df is empty, the subsequent inner merge for final_bureau_balance_features might
        # result in an empty df anyway, which is handled later.
            bureau_balance_loan_duration_categorised[col_name_to_ensure] = pd.Series(dtype='int')

dfs_to_merge = [bureau_balance_loan_duration_months,
bureau_balance_last_known_loan_status,
bureau_balance_nombre_delais_de_paiements,
bureau_balance_nombre_de_delais_de_paiements_par_categorie,
bureau_balance_duree_moyenne_delais_de_paiements,
bureau_balance_loan_duration_categorised]

In [14]:
bureau_balance_loan_duration_categorised.columns

Index(['SK_ID_BUREAU', 'YEAR_LOAN_DURATION', 'LOAN_TYPE_Long Term',
       'LOAN_TYPE_Short Term'],
      dtype='object')

# Getting list of columns for correction of agg_numeric for empty DataFrames

def agg_numeric(df, group_var, df_name, expected_columns_after_numeric_aggregation):
    if df.empty :
        agg = pd.DataFrame(columns=expected_columns_after_numeric_aggregation)
    else :
        for col in df:
            if col != group_var and 'SK_ID' in col:
                df = df.drop(columns = col)
        group_ids = df[group_var]
        numeric_df = df.select_dtypes('number')
        numeric_df[group_var] = group_ids
        agg = numeric_df.groupby(group_var).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()
        columns = [group_var]
        for var in agg.columns.levels[0]:
            if var != group_var:
                for stat in agg.columns.levels[1][:-1]:
                    columns.append('%s_%s_%s' % (df_name, var, stat))
        agg.columns = columns
    return agg

In [15]:
def agg_numeric(df, group_var, df_name):
    for col in df:
        if col != group_var and 'SK_ID' in col:
            df = df.drop(columns = col)
    group_ids = df[group_var]
    numeric_df = df.select_dtypes('number')
    numeric_df[group_var] = group_ids
    agg = numeric_df.groupby(group_var).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()
    columns = [group_var]
    for var in agg.columns.levels[0]:
        if var != group_var:
            for stat in agg.columns.levels[1][:-1]:
                columns.append('%s_%s_%s' % (df_name, var, stat))
    agg.columns = columns
    return agg

HomeCredit = HomeCredit_ini.copy()
POS_CASH_balance = POS_CASH_balance_ini.copy()
installments_payments = installments_payments_ini.copy()
previous_application = previous_application_ini.copy()
application_test = application_test_ini.copy()
application_train = application_train_ini.copy()
bureau_balance = bureau_balance_ini.copy()
bureau = bureau_ini.copy()
sample_submission = sample_submission_ini.copy()
credit_card_balance = credit_card_balance_ini.copy()
features_manual_and_func_from_first_three_with_app_train = features_manual_and_func_from_first_three_with_app_train_ini.copy()

In [16]:
num_features_with_function_bureau = agg_numeric(bureau.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'bureau')
num_features_with_function_bureau.columns

Index(['SK_ID_CURR', 'bureau_DAYS_CREDIT_count', 'bureau_DAYS_CREDIT_mean',
       'bureau_DAYS_CREDIT_max', 'bureau_DAYS_CREDIT_min',
       'bureau_DAYS_CREDIT_sum', 'bureau_CREDIT_DAY_OVERDUE_count',
       'bureau_CREDIT_DAY_OVERDUE_mean', 'bureau_CREDIT_DAY_OVERDUE_max',
       'bureau_CREDIT_DAY_OVERDUE_min', 'bureau_CREDIT_DAY_OVERDUE_sum',
       'bureau_DAYS_CREDIT_ENDDATE_count', 'bureau_DAYS_CREDIT_ENDDATE_mean',
       'bureau_DAYS_CREDIT_ENDDATE_max', 'bureau_DAYS_CREDIT_ENDDATE_min',
       'bureau_DAYS_CREDIT_ENDDATE_sum', 'bureau_DAYS_ENDDATE_FACT_count',
       'bureau_DAYS_ENDDATE_FACT_mean', 'bureau_DAYS_ENDDATE_FACT_max',
       'bureau_DAYS_ENDDATE_FACT_min', 'bureau_DAYS_ENDDATE_FACT_sum',
       'bureau_AMT_CREDIT_MAX_OVERDUE_count',
       'bureau_AMT_CREDIT_MAX_OVERDUE_mean',
       'bureau_AMT_CREDIT_MAX_OVERDUE_max',
       'bureau_AMT_CREDIT_MAX_OVERDUE_min',
       'bureau_AMT_CREDIT_MAX_OVERDUE_sum', 'bureau_CNT_CREDIT_PROLONG_count',
       'bureau_CNT_CR

In [17]:
num_features_with_function_bureau_balance = agg_numeric(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
num_features_with_function_bureau_balance.columns

Index(['SK_ID_BUREAU', 'bureau_balance_MONTHS_BALANCE_count',
       'bureau_balance_MONTHS_BALANCE_mean',
       'bureau_balance_MONTHS_BALANCE_max',
       'bureau_balance_MONTHS_BALANCE_min',
       'bureau_balance_MONTHS_BALANCE_sum'],
      dtype='object')

# Same for count_categorical

In [18]:
def count_categorical(df, group_var, df_name):
    categorical = pd.get_dummies(df.select_dtypes('object'))
    categorical[group_var] = df[group_var]
    categorical = categorical.groupby(group_var).agg(['sum', 'mean'])
    column_names = []
    for var in categorical.columns.levels[0]:
        for stat in ['count', 'count_norm']:
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    categorical.columns = column_names
    return categorical

In [19]:
cat_features_with_function_bureau = count_categorical(bureau, group_var = 'SK_ID_CURR', df_name = 'bureau')
cat_features_with_function_bureau.columns

Index(['bureau_CREDIT_ACTIVE_Active_count',
       'bureau_CREDIT_ACTIVE_Active_count_norm',
       'bureau_CREDIT_ACTIVE_Bad debt_count',
       'bureau_CREDIT_ACTIVE_Bad debt_count_norm',
       'bureau_CREDIT_ACTIVE_Closed_count',
       'bureau_CREDIT_ACTIVE_Closed_count_norm',
       'bureau_CREDIT_ACTIVE_Sold_count',
       'bureau_CREDIT_ACTIVE_Sold_count_norm',
       'bureau_CREDIT_CURRENCY_currency 1_count',
       'bureau_CREDIT_CURRENCY_currency 1_count_norm',
       'bureau_CREDIT_CURRENCY_currency 2_count',
       'bureau_CREDIT_CURRENCY_currency 2_count_norm',
       'bureau_CREDIT_CURRENCY_currency 3_count',
       'bureau_CREDIT_CURRENCY_currency 3_count_norm',
       'bureau_CREDIT_CURRENCY_currency 4_count',
       'bureau_CREDIT_CURRENCY_currency 4_count_norm',
       'bureau_CREDIT_TYPE_Another type of loan_count',
       'bureau_CREDIT_TYPE_Another type of loan_count_norm',
       'bureau_CREDIT_TYPE_Car loan_count',
       'bureau_CREDIT_TYPE_Car loan_count_norm'

In [20]:
cat_features_with_function_bureau_balance = count_categorical(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
cat_features_with_function_bureau_balance.columns

Index(['bureau_balance_STATUS_0_count', 'bureau_balance_STATUS_0_count_norm',
       'bureau_balance_STATUS_1_count', 'bureau_balance_STATUS_1_count_norm',
       'bureau_balance_STATUS_2_count', 'bureau_balance_STATUS_2_count_norm',
       'bureau_balance_STATUS_3_count', 'bureau_balance_STATUS_3_count_norm',
       'bureau_balance_STATUS_4_count', 'bureau_balance_STATUS_4_count_norm',
       'bureau_balance_STATUS_5_count', 'bureau_balance_STATUS_5_count_norm',
       'bureau_balance_STATUS_C_count', 'bureau_balance_STATUS_C_count_norm',
       'bureau_balance_STATUS_X_count', 'bureau_balance_STATUS_X_count_norm'],
      dtype='object')

# Getting columns list for the 4 other Datasets

In [21]:
previous_application_num_agg_SK_ID_CURR = agg_numeric(previous_application.drop(columns=['SK_ID_PREV']), group_var='SK_ID_CURR', df_name='previous_application')
previous_application_cat_agg_SK_ID_CURR = count_categorical(previous_application.drop(columns=['SK_ID_PREV']), group_var='SK_ID_CURR', df_name='previous_application')

In [22]:
previous_application_num_agg_SK_ID_CURR.columns

Index(['SK_ID_CURR', 'previous_application_AMT_ANNUITY_count',
       'previous_application_AMT_ANNUITY_mean',
       'previous_application_AMT_ANNUITY_max',
       'previous_application_AMT_ANNUITY_min',
       'previous_application_AMT_ANNUITY_sum',
       'previous_application_AMT_APPLICATION_count',
       'previous_application_AMT_APPLICATION_mean',
       'previous_application_AMT_APPLICATION_max',
       'previous_application_AMT_APPLICATION_min',
       'previous_application_AMT_APPLICATION_sum',
       'previous_application_AMT_CREDIT_count',
       'previous_application_AMT_CREDIT_mean',
       'previous_application_AMT_CREDIT_max',
       'previous_application_AMT_CREDIT_min',
       'previous_application_AMT_CREDIT_sum',
       'previous_application_AMT_DOWN_PAYMENT_count',
       'previous_application_AMT_DOWN_PAYMENT_mean',
       'previous_application_AMT_DOWN_PAYMENT_max',
       'previous_application_AMT_DOWN_PAYMENT_min',
       'previous_application_AMT_DOWN_PAYMENT_

In [23]:
print(list(previous_application_cat_agg_SK_ID_CURR.columns))

['previous_application_NAME_CONTRACT_TYPE_Cash loans_count', 'previous_application_NAME_CONTRACT_TYPE_Cash loans_count_norm', 'previous_application_NAME_CONTRACT_TYPE_Consumer loans_count', 'previous_application_NAME_CONTRACT_TYPE_Consumer loans_count_norm', 'previous_application_NAME_CONTRACT_TYPE_Revolving loans_count', 'previous_application_NAME_CONTRACT_TYPE_Revolving loans_count_norm', 'previous_application_NAME_CONTRACT_TYPE_XNA_count', 'previous_application_NAME_CONTRACT_TYPE_XNA_count_norm', 'previous_application_WEEKDAY_APPR_PROCESS_START_FRIDAY_count', 'previous_application_WEEKDAY_APPR_PROCESS_START_FRIDAY_count_norm', 'previous_application_WEEKDAY_APPR_PROCESS_START_MONDAY_count', 'previous_application_WEEKDAY_APPR_PROCESS_START_MONDAY_count_norm', 'previous_application_WEEKDAY_APPR_PROCESS_START_SATURDAY_count', 'previous_application_WEEKDAY_APPR_PROCESS_START_SATURDAY_count_norm', 'previous_application_WEEKDAY_APPR_PROCESS_START_SUNDAY_count', 'previous_application_WEEKDA

In [24]:
POS_CASH_balance_num_agg_SK_ID_CURR = agg_numeric(POS_CASH_balance.drop(columns=['SK_ID_PREV']), group_var='SK_ID_CURR', df_name='POS_CASH_balance')
POS_CASH_balance_cat_agg_SK_ID_CURR = count_categorical(POS_CASH_balance.drop(columns=['SK_ID_PREV']), group_var='SK_ID_CURR', df_name='POS_CASH_balance')

In [25]:
POS_CASH_balance_num_agg_SK_ID_CURR.columns

Index(['SK_ID_CURR', 'POS_CASH_balance_MONTHS_BALANCE_count',
       'POS_CASH_balance_MONTHS_BALANCE_mean',
       'POS_CASH_balance_MONTHS_BALANCE_max',
       'POS_CASH_balance_MONTHS_BALANCE_min',
       'POS_CASH_balance_MONTHS_BALANCE_sum',
       'POS_CASH_balance_CNT_INSTALMENT_count',
       'POS_CASH_balance_CNT_INSTALMENT_mean',
       'POS_CASH_balance_CNT_INSTALMENT_max',
       'POS_CASH_balance_CNT_INSTALMENT_min',
       'POS_CASH_balance_CNT_INSTALMENT_sum',
       'POS_CASH_balance_CNT_INSTALMENT_FUTURE_count',
       'POS_CASH_balance_CNT_INSTALMENT_FUTURE_mean',
       'POS_CASH_balance_CNT_INSTALMENT_FUTURE_max',
       'POS_CASH_balance_CNT_INSTALMENT_FUTURE_min',
       'POS_CASH_balance_CNT_INSTALMENT_FUTURE_sum',
       'POS_CASH_balance_SK_DPD_count', 'POS_CASH_balance_SK_DPD_mean',
       'POS_CASH_balance_SK_DPD_max', 'POS_CASH_balance_SK_DPD_min',
       'POS_CASH_balance_SK_DPD_sum', 'POS_CASH_balance_SK_DPD_DEF_count',
       'POS_CASH_balance_SK_DPD_DEF_

In [26]:
POS_CASH_balance_cat_agg_SK_ID_CURR.columns

Index(['POS_CASH_balance_NAME_CONTRACT_STATUS_Active_count',
       'POS_CASH_balance_NAME_CONTRACT_STATUS_Active_count_norm',
       'POS_CASH_balance_NAME_CONTRACT_STATUS_Amortized debt_count',
       'POS_CASH_balance_NAME_CONTRACT_STATUS_Amortized debt_count_norm',
       'POS_CASH_balance_NAME_CONTRACT_STATUS_Approved_count',
       'POS_CASH_balance_NAME_CONTRACT_STATUS_Approved_count_norm',
       'POS_CASH_balance_NAME_CONTRACT_STATUS_Canceled_count',
       'POS_CASH_balance_NAME_CONTRACT_STATUS_Canceled_count_norm',
       'POS_CASH_balance_NAME_CONTRACT_STATUS_Completed_count',
       'POS_CASH_balance_NAME_CONTRACT_STATUS_Completed_count_norm',
       'POS_CASH_balance_NAME_CONTRACT_STATUS_Demand_count',
       'POS_CASH_balance_NAME_CONTRACT_STATUS_Demand_count_norm',
       'POS_CASH_balance_NAME_CONTRACT_STATUS_Returned to the store_count',
       'POS_CASH_balance_NAME_CONTRACT_STATUS_Returned to the store_count_norm',
       'POS_CASH_balance_NAME_CONTRACT_STATUS_Signe

In [27]:
credit_card_balance_num_agg_SK_ID_CURR = agg_numeric(credit_card_balance.drop(columns=['SK_ID_PREV']), group_var='SK_ID_CURR', df_name='credit_card_balance')
credit_card_balance_cat_agg_SK_ID_CURR = count_categorical(credit_card_balance.drop(columns=['SK_ID_PREV']), group_var='SK_ID_CURR', df_name='credit_card_balance')

In [28]:
print(list(credit_card_balance_num_agg_SK_ID_CURR.columns))

['SK_ID_CURR', 'credit_card_balance_MONTHS_BALANCE_count', 'credit_card_balance_MONTHS_BALANCE_mean', 'credit_card_balance_MONTHS_BALANCE_max', 'credit_card_balance_MONTHS_BALANCE_min', 'credit_card_balance_MONTHS_BALANCE_sum', 'credit_card_balance_AMT_BALANCE_count', 'credit_card_balance_AMT_BALANCE_mean', 'credit_card_balance_AMT_BALANCE_max', 'credit_card_balance_AMT_BALANCE_min', 'credit_card_balance_AMT_BALANCE_sum', 'credit_card_balance_AMT_CREDIT_LIMIT_ACTUAL_count', 'credit_card_balance_AMT_CREDIT_LIMIT_ACTUAL_mean', 'credit_card_balance_AMT_CREDIT_LIMIT_ACTUAL_max', 'credit_card_balance_AMT_CREDIT_LIMIT_ACTUAL_min', 'credit_card_balance_AMT_CREDIT_LIMIT_ACTUAL_sum', 'credit_card_balance_AMT_DRAWINGS_ATM_CURRENT_count', 'credit_card_balance_AMT_DRAWINGS_ATM_CURRENT_mean', 'credit_card_balance_AMT_DRAWINGS_ATM_CURRENT_max', 'credit_card_balance_AMT_DRAWINGS_ATM_CURRENT_min', 'credit_card_balance_AMT_DRAWINGS_ATM_CURRENT_sum', 'credit_card_balance_AMT_DRAWINGS_CURRENT_count', 'cr

In [29]:
credit_card_balance_cat_agg_SK_ID_CURR.columns

Index(['credit_card_balance_NAME_CONTRACT_STATUS_Active_count',
       'credit_card_balance_NAME_CONTRACT_STATUS_Active_count_norm',
       'credit_card_balance_NAME_CONTRACT_STATUS_Approved_count',
       'credit_card_balance_NAME_CONTRACT_STATUS_Approved_count_norm',
       'credit_card_balance_NAME_CONTRACT_STATUS_Completed_count',
       'credit_card_balance_NAME_CONTRACT_STATUS_Completed_count_norm',
       'credit_card_balance_NAME_CONTRACT_STATUS_Demand_count',
       'credit_card_balance_NAME_CONTRACT_STATUS_Demand_count_norm',
       'credit_card_balance_NAME_CONTRACT_STATUS_Refused_count',
       'credit_card_balance_NAME_CONTRACT_STATUS_Refused_count_norm',
       'credit_card_balance_NAME_CONTRACT_STATUS_Sent proposal_count',
       'credit_card_balance_NAME_CONTRACT_STATUS_Sent proposal_count_norm',
       'credit_card_balance_NAME_CONTRACT_STATUS_Signed_count',
       'credit_card_balance_NAME_CONTRACT_STATUS_Signed_count_norm'],
      dtype='object')

In [30]:
installments_payments_num_agg_SK_ID_CURR = agg_numeric(installments_payments.drop(columns=['SK_ID_PREV']), group_var='SK_ID_CURR', df_name='credit_card_balance')
installments_payments_num_agg_SK_ID_CURR.columns

Index(['SK_ID_CURR', 'credit_card_balance_NUM_INSTALMENT_VERSION_count',
       'credit_card_balance_NUM_INSTALMENT_VERSION_mean',
       'credit_card_balance_NUM_INSTALMENT_VERSION_max',
       'credit_card_balance_NUM_INSTALMENT_VERSION_min',
       'credit_card_balance_NUM_INSTALMENT_VERSION_sum',
       'credit_card_balance_NUM_INSTALMENT_NUMBER_count',
       'credit_card_balance_NUM_INSTALMENT_NUMBER_mean',
       'credit_card_balance_NUM_INSTALMENT_NUMBER_max',
       'credit_card_balance_NUM_INSTALMENT_NUMBER_min',
       'credit_card_balance_NUM_INSTALMENT_NUMBER_sum',
       'credit_card_balance_DAYS_INSTALMENT_count',
       'credit_card_balance_DAYS_INSTALMENT_mean',
       'credit_card_balance_DAYS_INSTALMENT_max',
       'credit_card_balance_DAYS_INSTALMENT_min',
       'credit_card_balance_DAYS_INSTALMENT_sum',
       'credit_card_balance_DAYS_ENTRY_PAYMENT_count',
       'credit_card_balance_DAYS_ENTRY_PAYMENT_mean',
       'credit_card_balance_DAYS_ENTRY_PAYMENT_max',


In [31]:
# Création des features à l'aide des deux fonctions d'aggrégation
num_features_with_function_bureau = agg_numeric(bureau.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'bureau')
cat_features_with_function_bureau = count_categorical(bureau, group_var = 'SK_ID_CURR', df_name = 'bureau')

num_features_with_function_bureau_balance = agg_numeric(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
cat_features_with_function_bureau_balance = count_categorical(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')

num_and_cat_features_with_function_bureau_balance = pd.merge(num_features_with_function_bureau_balance, cat_features_with_function_bureau_balance, how='inner', on='SK_ID_BUREAU')
ids_for_agg = bureau[['SK_ID_CURR', 'SK_ID_BUREAU']]
num_and_cat_features_with_function_bureau_balance_with_curr = pd.merge(left=ids_for_agg, right=num_and_cat_features_with_function_bureau_balance, how='inner').sort_values(by='SK_ID_BUREAU', ascending=True)
num_and_cat_features_with_function_bureau_balance_with_curr_agg = agg_numeric(num_and_cat_features_with_function_bureau_balance_with_curr.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'client')
print(list(num_and_cat_features_with_function_bureau_balance_with_curr_agg.columns))

['SK_ID_CURR', 'client_bureau_balance_MONTHS_BALANCE_count_count', 'client_bureau_balance_MONTHS_BALANCE_count_mean', 'client_bureau_balance_MONTHS_BALANCE_count_max', 'client_bureau_balance_MONTHS_BALANCE_count_min', 'client_bureau_balance_MONTHS_BALANCE_count_sum', 'client_bureau_balance_MONTHS_BALANCE_mean_count', 'client_bureau_balance_MONTHS_BALANCE_mean_mean', 'client_bureau_balance_MONTHS_BALANCE_mean_max', 'client_bureau_balance_MONTHS_BALANCE_mean_min', 'client_bureau_balance_MONTHS_BALANCE_mean_sum', 'client_bureau_balance_MONTHS_BALANCE_max_count', 'client_bureau_balance_MONTHS_BALANCE_max_mean', 'client_bureau_balance_MONTHS_BALANCE_max_max', 'client_bureau_balance_MONTHS_BALANCE_max_min', 'client_bureau_balance_MONTHS_BALANCE_max_sum', 'client_bureau_balance_MONTHS_BALANCE_min_count', 'client_bureau_balance_MONTHS_BALANCE_min_mean', 'client_bureau_balance_MONTHS_BALANCE_min_max', 'client_bureau_balance_MONTHS_BALANCE_min_min', 'client_bureau_balance_MONTHS_BALANCE_min_sum'

manual_final_bureau_balance_features_with_curr_agg = agg_numeric(final_bureau_balance_features_with_curr.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'client_bureau_balance')
print(list(manual_final_bureau_balance_features_with_curr_agg.columns))

# Sep_test

In [32]:
from functools import reduce

# Création des features "manuelles"
bureau_balance_loan_duration_months = bureau_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].count().reset_index()
bureau_balance_loan_duration_months.rename(columns = {'MONTHS_BALANCE' : 'MONTHS_LOAN_DURATION'}, inplace=True)

bureau_balance_last_known_loan_status = bureau_balance.sort_values(by='MONTHS_BALANCE', ascending=False).groupby('SK_ID_BUREAU').first().reset_index()[['SK_ID_BUREAU', 'STATUS']]
#bureau_balance_last_known_loan_status = pd.get_dummies(bureau_balance_last_known_loan_status, dtype=int)
all_expected_status_categories = ['0', '1', '2', '3', '4', '5', 'C', 'X'] # These are the raw STATUS values
for cat_col_name in all_expected_status_categories:
    if cat_col_name not in bureau_balance_last_known_loan_status.columns:
        bureau_balance_last_known_loan_status[cat_col_name] = 0

bureau_balance_for_dpd_flag = bureau_balance.copy()
non_dpd_statuses = ['C', 'X', '0']
dpd_flagged = ~bureau_balance_for_dpd_flag['STATUS'].isin(non_dpd_statuses) #Cree True/False value en fonction de la condition ici dpd --> True
bureau_balance_for_dpd_flag['DPD_FLAG'] = dpd_flagged.astype(int)
bureau_balance_nombre_delais_de_paiements = bureau_balance_for_dpd_flag.groupby(by='SK_ID_BUREAU')['DPD_FLAG'].sum().reset_index()

bureau_balance_nombre_de_delais_de_paiements_par_categorie = bureau_balance.groupby('SK_ID_BUREAU')['STATUS'].value_counts().unstack(fill_value=0).reset_index()
bureau_balance_nombre_de_delais_de_paiements_par_categorie.columns.name = None

Values_to_map = {'C':0, 'X':0, '0':0, '1':15, '2':45, '3':75, '4':105, '5':120}
bureau_balance_for_mean_delay_calc = bureau_balance.copy()
bureau_balance_for_mean_delay_calc['MEAN_DAYS_PAST_DUE'] = bureau_balance_for_mean_delay_calc['STATUS'].map(Values_to_map).fillna('Unknown')
bureau_balance_duree_moyenne_delais_de_paiements = bureau_balance_for_mean_delay_calc.groupby('SK_ID_BUREAU')['MEAN_DAYS_PAST_DUE'].mean().reset_index()

bureau_balance_loan_duration_categorised = bureau_balance.sort_values(by='MONTHS_BALANCE', ascending=True).groupby('SK_ID_BUREAU').first().reset_index()
bureau_balance_loan_duration_categorised['YEAR_LOAN_DURATION'] = round(bureau_balance_loan_duration_categorised['MONTHS_BALANCE']/(-12), 1)
bureau_balance_loan_duration_categorised['LOAN_TYPE'] = np.where(
    bureau_balance_loan_duration_categorised['YEAR_LOAN_DURATION'] >= 5, # Condition
    'Long Term',                                             # Value if True
    'Short Term'                                             # Value if False
)
bureau_balance_loan_duration_categorised = bureau_balance_loan_duration_categorised.drop(columns=['MONTHS_BALANCE', 'STATUS'], axis=0)
bureau_balance_loan_duration_categorised = pd.get_dummies(
    bureau_balance_loan_duration_categorised, # This DataFrame ALREADY has 'SK_ID_BUREAU'
    columns=['LOAN_TYPE'], # This will only dummify LOAN_TYPE and not risk adding another SK_ID_BUREAU
    dtype=int
    # prefix='LOAN_TYPE' # You should add a prefix here if not already done
)

# Failproofnet loan type
expected_loan_type_dummies = ['LOAN_TYPE_Long Term', 'LOAN_TYPE_Short Term']
for col_name_to_ensure in expected_loan_type_dummies:
    if col_name_to_ensure not in bureau_balance_loan_duration_categorised.columns:
    # If the DataFrame is not empty, assign 0. 
    # If it's empty, assign an empty Series of the correct dtype.
    # This ensures the column exists for schema consistency in merges.
        if not bureau_balance_loan_duration_categorised.empty:
            bureau_balance_loan_duration_categorised[col_name_to_ensure] = 0
        else:
        # If bureau_balance_loan_duration_categorised is truly empty (no rows, no SK_ID_BUREAU),
        # creating just an empty Series might still cause issues if SK_ID_BUREAU is needed
        # for the merge into final_bureau_balance_features.
        # However, the goal here is to prevent the KeyError when these columns are *selected*.
        # If this df is empty, the subsequent inner merge for final_bureau_balance_features might
        # result in an empty df anyway, which is handled later.
            bureau_balance_loan_duration_categorised[col_name_to_ensure] = pd.Series(dtype='int')

dfs_to_merge = [bureau_balance_loan_duration_months,
bureau_balance_last_known_loan_status,
bureau_balance_nombre_delais_de_paiements,
bureau_balance_nombre_de_delais_de_paiements_par_categorie,
bureau_balance_duree_moyenne_delais_de_paiements,
bureau_balance_loan_duration_categorised]

# We add a step to make sure the merge key is present to prevent errors in the reduce function

expected_features_for_bureau_balance_loan_duration_months = ['SK_ID_BUREAU', 'MONTHS_LOAN_DURATION']
expected_features_for_bureau_balance_last_known_loan_status = ['SK_ID_BUREAU', 'STATUS', '0', '1', '2', '3', '4', '5', 'C', 'X']
expected_features_for_bureau_balance_nombre_delais_de_paiements = ['SK_ID_BUREAU', 'DPD_FLAG']
expected_features_for_bureau_balance_nombre_de_delais_de_paiements_par_categorie = ['SK_ID_BUREAU', '0', '1', '2', '3', '4', '5', 'C', 'X']
expected_features_for_bureau_balance_duree_moyenne_delais_de_paiements = ['SK_ID_BUREAU', 'MEAN_DAYS_PAST_DUE']
expected_features_for_bureau_balance_loan_duration_categorised = ['SK_ID_BUREAU', 'YEAR_LOAN_DURATION', 'LOAN_TYPE_Long Term', 'LOAN_TYPE_Short Term']

list_of_expected_features_per_df = [
    expected_features_for_bureau_balance_loan_duration_months, expected_features_for_bureau_balance_last_known_loan_status, 
    expected_features_for_bureau_balance_nombre_delais_de_paiements, expected_features_for_bureau_balance_nombre_de_delais_de_paiements_par_categorie, 
    expected_features_for_bureau_balance_duree_moyenne_delais_de_paiements, expected_features_for_bureau_balance_loan_duration_categorised
    ]

validated_dfs_to_merge = []

for df_unchecked, expected_col_list in zip(dfs_to_merge, list_of_expected_features_per_df): #utiliser zip plutot que if and sinon ca va pas faire correspondre correctement les deux conditions par exemple ça va mettre df 2 avec liste de colonne 1
    if df_unchecked.empty :
        df_corrected = pd.DataFrame(columns=expected_col_list)
        validated_dfs_to_merge.append(df_corrected)
    else :
        validated_dfs_to_merge.append(df_unchecked)

merge_key = 'SK_ID_BUREAU'

merge_function = lambda left_df, right_df: pd.merge(left_df, right_df, on=merge_key, how='inner')
final_bureau_balance_features = reduce(merge_function, validated_dfs_to_merge)
#final_bureau_balance_features = final_bureau_balance_features.drop(columns=['0', 'C', 'X'])


bureau_for_credit_status = bureau.copy()
bureau_number_of_type_of_credits = bureau.groupby('SK_ID_CURR')['CREDIT_ACTIVE'].value_counts().unstack(fill_value=0).reset_index()
bureau_number_of_type_of_credits.columns.name = None

bureau_for_encoding_credit_types = bureau[['SK_ID_CURR', 'CREDIT_TYPE']]
bureau_credit_type_encoded = pd.get_dummies(
    bureau_for_encoding_credit_types, 
    columns=['CREDIT_TYPE'], 
    dtype=int
    )
bureau_liste_colonnes_encodees = bureau_credit_type_encoded.columns
bureau_liste_colonnes_a_agreger = [col for col in bureau_liste_colonnes_encodees if col != 'SK_ID_CURR']
bureau_credit_type_encoded_and_aggregated = bureau_credit_type_encoded.groupby('SK_ID_CURR')[bureau_liste_colonnes_a_agreger].sum().reset_index()

bureau_for_max_amount_overdue = bureau[['SK_ID_CURR', 'AMT_CREDIT_MAX_OVERDUE']]
bureau_for_max_amount_overdue.fillna(value=0, inplace=True)
bureau_for_max_amount_overdue = bureau_for_max_amount_overdue.groupby('SK_ID_CURR')['AMT_CREDIT_MAX_OVERDUE'].max().reset_index()

bureau_for_mean_amount_overdue_across_loans = bureau[['SK_ID_CURR', 'AMT_CREDIT_SUM_OVERDUE']]
bureau_for_mean_amount_overdue_across_loans = bureau_for_mean_amount_overdue_across_loans.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_OVERDUE'].mean().reset_index()

bureau_for_credit_prolonged_count = bureau[['SK_ID_CURR', 'CNT_CREDIT_PROLONG']]
bureau_for_credit_prolonged_count = bureau_for_credit_prolonged_count.groupby('SK_ID_CURR')['CNT_CREDIT_PROLONG'].sum().reset_index()

bureau_for_proportions_repaid = bureau[['SK_ID_CURR', 'SK_ID_BUREAU', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM']]
bureau_for_proportions_repaid.dropna(subset='AMT_CREDIT_SUM', inplace=True)
bureau_for_proportions_repaid['AMT_CREDIT_SUM_DEBT_IS_MISSING'] = bureau_for_proportions_repaid['AMT_CREDIT_SUM_DEBT'].isnull().astype(int)
mask_active_missing_debt = bureau_for_proportions_repaid['AMT_CREDIT_SUM_DEBT'].isnull()
bureau_for_proportions_repaid.loc[mask_active_missing_debt, 'AMT_CREDIT_SUM_DEBT'] = bureau_for_proportions_repaid.loc[mask_active_missing_debt, 'AMT_CREDIT_SUM']
bureau_for_proportions_repaid_with_nan_values_treated = bureau_for_proportions_repaid.copy()
bureau_for_proportions_repaid_with_nan_values_treated = bureau_for_proportions_repaid_with_nan_values_treated.groupby('SK_ID_CURR')[['AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM']].sum().reset_index()
bureau_for_proportions_repaid_with_nan_values_treated['PROPORTION_REPAID'] = np.nan
# Rule 1: SUM = 0 and DEBT = 0  => Repaid = 0
mask_zero_both = (bureau_for_proportions_repaid_with_nan_values_treated['AMT_CREDIT_SUM'] == 0) & (bureau_for_proportions_repaid_with_nan_values_treated['AMT_CREDIT_SUM_DEBT'] == 0)
bureau_for_proportions_repaid_with_nan_values_treated.loc[mask_zero_both, 'PROPORTION_REPAID'] = 1.0
# Rule 2: DEBT < 0 => Repaid = 1
mask_neg_debt = bureau_for_proportions_repaid_with_nan_values_treated['AMT_CREDIT_SUM_DEBT'] < 0
bureau_for_proportions_repaid_with_nan_values_treated.loc[mask_neg_debt, 'PROPORTION_REPAID'] = 1.0
# Rule 4: DEBT>0 and SUM = 0
mask_pos_debt_null_sum = (bureau_for_proportions_repaid_with_nan_values_treated['AMT_CREDIT_SUM'] == 0) & (bureau_for_proportions_repaid_with_nan_values_treated['AMT_CREDIT_SUM_DEBT'] > 0)
bureau_for_proportions_repaid_with_nan_values_treated.loc[mask_pos_debt_null_sum, 'PROPORTION_REPAID'] = 0
# Rule 4: All other cases where SUM > 0
mask_calculate = (bureau_for_proportions_repaid_with_nan_values_treated['AMT_CREDIT_SUM'] > 0) & (bureau_for_proportions_repaid_with_nan_values_treated['PROPORTION_REPAID'].isnull()) # Only calculate where not set yet
bureau_for_proportions_repaid_with_nan_values_treated.loc[mask_calculate, 'PROPORTION_REPAID'] = 1.0 - (bureau_for_proportions_repaid_with_nan_values_treated.loc[mask_calculate, 'AMT_CREDIT_SUM_DEBT'] / bureau_for_proportions_repaid_with_nan_values_treated.loc[mask_calculate, 'AMT_CREDIT_SUM'])

bureau_for_deadline_in_the_past = bureau[['SK_ID_CURR', 'CREDIT_ACTIVE', 'DAYS_CREDIT_ENDDATE']].copy()
bureau_for_deadline_in_the_past['LOAN_ACTIVE_PAST_DEADLINE'] = np.where(
    (bureau_for_deadline_in_the_past['CREDIT_ACTIVE'] == 'Active') & (bureau_for_deadline_in_the_past['DAYS_CREDIT_ENDDATE'] < 0), # Condition
    1,                                             # Value if True
    0                                             # Value if False
)
bureau_for_deadline_in_the_past = bureau_for_deadline_in_the_past.groupby('SK_ID_CURR')['LOAN_ACTIVE_PAST_DEADLINE'].sum().reset_index()

bureau_for_mean_days_spent_overdue = bureau[['SK_ID_CURR', 'CREDIT_DAY_OVERDUE']]
bureau_for_mean_days_spent_overdue = bureau_for_mean_days_spent_overdue.groupby('SK_ID_CURR')['CREDIT_DAY_OVERDUE'].mean().reset_index()

dfs_to_merge = [bureau_number_of_type_of_credits,
bureau_credit_type_encoded_and_aggregated,
bureau_for_max_amount_overdue,
bureau_for_mean_amount_overdue_across_loans,
bureau_for_credit_prolonged_count,
bureau_for_proportions_repaid_with_nan_values_treated,
bureau_for_deadline_in_the_past,
bureau_for_mean_days_spent_overdue]

merge_key = 'SK_ID_CURR'

merge_function = lambda left_df, right_df: pd.merge(left_df, right_df, on=merge_key, how='inner')
final_bureau_features = reduce(merge_function, dfs_to_merge)

ids_for_agg = bureau[['SK_ID_CURR', 'SK_ID_BUREAU']]
final_bureau_balance_features_with_curr = pd.merge(left=ids_for_agg, right=final_bureau_balance_features, how='inner')
all_expected_status_categories = ['0', '1', '2', '3', '4', '5', 'C', 'X'] # These are the raw STATUS values
for cat_col_name in all_expected_status_categories:
    if cat_col_name not in final_bureau_balance_features_with_curr.columns:
        final_bureau_balance_features_with_curr[cat_col_name] = 0
final_bureau_balance_features_with_curr_mean = final_bureau_balance_features_with_curr.groupby('SK_ID_CURR')[['MONTHS_LOAN_DURATION', 'DPD_FLAG', '1', '2', '3', '4', '5', 'MEAN_DAYS_PAST_DUE', 'YEAR_LOAN_DURATION']].mean().reset_index()
rename_mean_dict = {col: 'MEAN_' + str(col) for col in final_bureau_balance_features_with_curr_mean.columns if col != 'SK_ID_CURR'}
final_bureau_balance_features_with_curr_mean.rename(columns=rename_mean_dict, inplace=True)



all_expected_status_categories = ['STATUS_0','STATUS_1', 'STATUS_2', 'STATUS_3', 'STATUS_4', 'STATUS_5', 'STATUS_C','STATUS_X'] # These are the raw STATUS values
for cat_col_name in all_expected_status_categories:
    if cat_col_name not in final_bureau_balance_features_with_curr.columns:
        final_bureau_balance_features_with_curr[cat_col_name] = 0

final_bureau_balance_features_with_curr_sum = final_bureau_balance_features_with_curr.groupby('SK_ID_CURR')[['STATUS_0','STATUS_1', 'STATUS_2', 'STATUS_3', 'STATUS_4', 'STATUS_5', 'STATUS_C','STATUS_X', 'DPD_FLAG', '1', '2', '3', '4', '5', 'LOAN_TYPE_Long Term', 'LOAN_TYPE_Short Term']].sum().reset_index()
rename_sum_dict = {col: 'SUM_' + str(col) for col in final_bureau_balance_features_with_curr_sum.columns if col != 'SK_ID_CURR'}
final_bureau_balance_features_with_curr_sum.rename(columns=rename_sum_dict, inplace=True)

final_bureau_balance_features_with_curr_agg = pd.merge(left=final_bureau_balance_features_with_curr_mean, right=final_bureau_balance_features_with_curr_sum, how='inner', on='SK_ID_CURR')
features_bureau_bureau_balance = pd.merge(left=final_bureau_features, right=final_bureau_balance_features_with_curr_agg, how='inner', on='SK_ID_CURR')

# Création des features à l'aide des deux fonctions d'aggrégation
num_features_with_function_bureau = agg_numeric(bureau.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'bureau')
cat_features_with_function_bureau = count_categorical(bureau, group_var = 'SK_ID_CURR', df_name = 'bureau')

num_features_with_function_bureau_balance = agg_numeric(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
cat_features_with_function_bureau_balance = count_categorical(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')

num_and_cat_features_with_function_bureau_balance = pd.merge(num_features_with_function_bureau_balance, cat_features_with_function_bureau_balance, how='inner', on='SK_ID_BUREAU')
ids_for_agg = bureau[['SK_ID_CURR', 'SK_ID_BUREAU']]
num_and_cat_features_with_function_bureau_balance_with_curr = pd.merge(left=ids_for_agg, right=num_and_cat_features_with_function_bureau_balance, how='inner').sort_values(by='SK_ID_BUREAU', ascending=True)



num_and_cat_features_with_function_bureau_balance_with_curr_agg = agg_numeric(num_and_cat_features_with_function_bureau_balance_with_curr.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'client')



manual_final_bureau_balance_features_with_curr_agg = agg_numeric(final_bureau_balance_features_with_curr.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'client_bureau_balance')

D:\Temp\TEMP\ipykernel_13580\2799184793.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bureau_for_max_amount_overdue.fillna(value=0, inplace=True)
D:\Temp\TEMP\ipykernel_13580\2799184793.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bureau_for_proportions_repaid.dropna(subset='AMT_CREDIT_SUM', inplace=True)
D:\Temp\TEMP\ipykernel_13580\2799184793.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [33]:
manual_final_bureau_balance_features_with_curr_agg


,SK_ID_CURR,client_bureau_balance_MONTHS_LOAN_DURATION_count,client_bureau_balance_MONTHS_LOAN_DURATION_mean,client_bureau_balance_MONTHS_LOAN_DURATION_max,client_bureau_balance_MONTHS_LOAN_DURATION_min,client_bureau_balance_MONTHS_LOAN_DURATION_sum,client_bureau_balance_0_x_count,client_bureau_balance_0_x_mean,client_bureau_balance_0_x_max,client_bureau_balance_0_x_min,...,client_bureau_balance_STATUS_C_count,client_bureau_balance_STATUS_C_mean,client_bureau_balance_STATUS_C_max,client_bureau_balance_STATUS_C_min,client_bureau_balance_STATUS_C_sum,client_bureau_balance_STATUS_X_count,client_bureau_balance_STATUS_X_mean,client_bureau_balance_STATUS_X_max,client_bureau_balance_STATUS_X_min,client_bureau_balance_STATUS_X_sum
0,100001,7,24.571429,52,2,172,7,0.0,0,0,...,7,0.0,0,0,0,7,0.0,0,0,0
1,100002,8,13.750000,22,4,110,8,0.0,0,0,...,8,0.0,0,0,0,8,0.0,0,0,0
2,100005,3,7.000000,13,3,21,3,0.0,0,0,...,3,0.0,0,0,0,3,0.0,0,0,0
3,100010,2,36.000000,36,36,72,2,0.0,0,0,...,2,0.0,0,0,0,2,0.0,0,0,0
4,100013,4,57.500000,69,40,230,4,0.0,0,0,...,4,0.0,0,0,0,4,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134537,456247,11,29.090909,82,10,320,11,0.0,0,0,...,11,0.0,0,0,0,11,0.0,0,0,0
134538,456250,3,29.000000,33,26,87,3,0.0,0,0,...,3,0.0,0,0,0,3,0.0,0,0,0
134539,456253,4,29.250000,31,24,117,4,0.0,0,0,...,4,0.0,0,0,0,4,0.0,0,0,0
134540,456254,1,37.000000,37,37,37,1,0.0,0,0,...,1,0.0,0,0,0,1,0.0,0,0,0


In [34]:
print(list(manual_final_bureau_balance_features_with_curr_agg.columns))

['SK_ID_CURR', 'client_bureau_balance_MONTHS_LOAN_DURATION_count', 'client_bureau_balance_MONTHS_LOAN_DURATION_mean', 'client_bureau_balance_MONTHS_LOAN_DURATION_max', 'client_bureau_balance_MONTHS_LOAN_DURATION_min', 'client_bureau_balance_MONTHS_LOAN_DURATION_sum', 'client_bureau_balance_0_x_count', 'client_bureau_balance_0_x_mean', 'client_bureau_balance_0_x_max', 'client_bureau_balance_0_x_min', 'client_bureau_balance_0_x_sum', 'client_bureau_balance_1_x_count', 'client_bureau_balance_1_x_mean', 'client_bureau_balance_1_x_max', 'client_bureau_balance_1_x_min', 'client_bureau_balance_1_x_sum', 'client_bureau_balance_2_x_count', 'client_bureau_balance_2_x_mean', 'client_bureau_balance_2_x_max', 'client_bureau_balance_2_x_min', 'client_bureau_balance_2_x_sum', 'client_bureau_balance_3_x_count', 'client_bureau_balance_3_x_mean', 'client_bureau_balance_3_x_max', 'client_bureau_balance_3_x_min', 'client_bureau_balance_3_x_sum', 'client_bureau_balance_4_x_count', 'client_bureau_balance_4_

# Getting final_bureau_manual_features of columns

In [35]:
bureau = bureau_ini.copy()
bureau_balance = bureau_balance_ini.copy()

In [36]:
# --- Aggrégation de bureau et bureau_balance ---

if bureau_balance.shape == (0,0) :
    bureau_balance = pd.DataFrame(columns=initial_expected_columns_bureau_balance)

if bureau.shape == (0,0) :
    bureau= pd.DataFrame(columns=initial_expected_columns_bureau)

# Création des features "manuelles"
bureau_balance_loan_duration_months = bureau_balance.groupby('SK_ID_BUREAU')['MONTHS_BALANCE'].count().reset_index()
bureau_balance_loan_duration_months.rename(columns = {'MONTHS_BALANCE' : 'MONTHS_LOAN_DURATION'}, inplace=True)

bureau_balance_last_known_loan_status = bureau_balance.sort_values(by='MONTHS_BALANCE', ascending=False).groupby('SK_ID_BUREAU').first().reset_index()[['SK_ID_BUREAU', 'STATUS']]
#bureau_balance_last_known_loan_status = pd.get_dummies(bureau_balance_last_known_loan_status, dtype=int)
all_expected_status_categories = ['0', '1', '2', '3', '4', '5', 'C', 'X'] # These are the raw STATUS values
for cat_col_name in all_expected_status_categories:
    if cat_col_name not in bureau_balance_last_known_loan_status.columns:
        bureau_balance_last_known_loan_status[cat_col_name] = 0

bureau_balance_for_dpd_flag = bureau_balance.copy()
non_dpd_statuses = ['C', 'X', '0']
dpd_flagged = ~bureau_balance_for_dpd_flag['STATUS'].isin(non_dpd_statuses) #Cree True/False value en fonction de la condition ici dpd --> True
bureau_balance_for_dpd_flag['DPD_FLAG'] = dpd_flagged.astype(int)
bureau_balance_nombre_delais_de_paiements = bureau_balance_for_dpd_flag.groupby(by='SK_ID_BUREAU')['DPD_FLAG'].sum().reset_index()

bureau_balance_nombre_de_delais_de_paiements_par_categorie = bureau_balance.groupby('SK_ID_BUREAU')['STATUS'].value_counts().unstack(fill_value=0).reset_index()
bureau_balance_nombre_de_delais_de_paiements_par_categorie.columns.name = None

Values_to_map = {'C':0, 'X':0, '0':0, '1':15, '2':45, '3':75, '4':105, '5':120}
bureau_balance_for_mean_delay_calc = bureau_balance.copy()
bureau_balance_for_mean_delay_calc['MEAN_DAYS_PAST_DUE'] = bureau_balance_for_mean_delay_calc['STATUS'].map(Values_to_map).fillna('Unknown')
bureau_balance_duree_moyenne_delais_de_paiements = bureau_balance_for_mean_delay_calc.groupby('SK_ID_BUREAU')['MEAN_DAYS_PAST_DUE'].mean().reset_index()

bureau_balance_loan_duration_categorised = bureau_balance.sort_values(by='MONTHS_BALANCE', ascending=True).groupby('SK_ID_BUREAU').first().reset_index()
bureau_balance_loan_duration_categorised['YEAR_LOAN_DURATION'] = round(bureau_balance_loan_duration_categorised['MONTHS_BALANCE']/(-12), 1)
bureau_balance_loan_duration_categorised['LOAN_TYPE'] = np.where(
    bureau_balance_loan_duration_categorised['YEAR_LOAN_DURATION'] >= 5, # Condition
    'Long Term',                                             # Value if True
    'Short Term'                                             # Value if False
)
bureau_balance_loan_duration_categorised = bureau_balance_loan_duration_categorised.drop(columns=['MONTHS_BALANCE', 'STATUS'], axis=0)
bureau_balance_loan_duration_categorised = pd.get_dummies(
    bureau_balance_loan_duration_categorised, # This DataFrame ALREADY has 'SK_ID_BUREAU'
    columns=['LOAN_TYPE'], # This will only dummify LOAN_TYPE and not risk adding another SK_ID_BUREAU
    dtype=int
    # prefix='LOAN_TYPE' # You should add a prefix here if not already done
)

# Failproofnet loan type
expected_loan_type_dummies = ['LOAN_TYPE_Long Term', 'LOAN_TYPE_Short Term']
for col_name_to_ensure in expected_loan_type_dummies:
    if col_name_to_ensure not in bureau_balance_loan_duration_categorised.columns:
    # If the DataFrame is not empty, assign 0. 
    # If it's empty, assign an empty Series of the correct dtype.
    # This ensures the column exists for schema consistency in merges.
        if not bureau_balance_loan_duration_categorised.empty:
            bureau_balance_loan_duration_categorised[col_name_to_ensure] = 0
        else:
        # If bureau_balance_loan_duration_categorised is truly empty (no rows, no SK_ID_BUREAU),
        # creating just an empty Series might still cause issues if SK_ID_BUREAU is needed
        # for the merge into final_bureau_balance_features.
        # However, the goal here is to prevent the KeyError when these columns are *selected*.
        # If this df is empty, the subsequent inner merge for final_bureau_balance_features might
        # result in an empty df anyway, which is handled later.
            bureau_balance_loan_duration_categorised[col_name_to_ensure] = pd.Series(dtype='int')

dfs_to_merge = [bureau_balance_loan_duration_months,
bureau_balance_last_known_loan_status,
bureau_balance_nombre_delais_de_paiements,
bureau_balance_nombre_de_delais_de_paiements_par_categorie,
bureau_balance_duree_moyenne_delais_de_paiements,
bureau_balance_loan_duration_categorised]

# We add a step to make sure the merge key is present to prevent errors in the reduce function

expected_features_for_bureau_balance_loan_duration_months = ['SK_ID_BUREAU', 'MONTHS_LOAN_DURATION']
expected_features_for_bureau_balance_last_known_loan_status = ['SK_ID_BUREAU', 'STATUS', '0', '1', '2', '3', '4', '5', 'C', 'X']
expected_features_for_bureau_balance_nombre_delais_de_paiements = ['SK_ID_BUREAU', 'DPD_FLAG']
expected_features_for_bureau_balance_nombre_de_delais_de_paiements_par_categorie = ['SK_ID_BUREAU', '0', '1', '2', '3', '4', '5', 'C', 'X']
expected_features_for_bureau_balance_duree_moyenne_delais_de_paiements = ['SK_ID_BUREAU', 'MEAN_DAYS_PAST_DUE']
expected_features_for_bureau_balance_loan_duration_categorised = ['SK_ID_BUREAU', 'YEAR_LOAN_DURATION', 'LOAN_TYPE_Long Term', 'LOAN_TYPE_Short Term']

list_of_expected_features_per_df = [
    expected_features_for_bureau_balance_loan_duration_months, expected_features_for_bureau_balance_last_known_loan_status, 
    expected_features_for_bureau_balance_nombre_delais_de_paiements, expected_features_for_bureau_balance_nombre_de_delais_de_paiements_par_categorie, 
    expected_features_for_bureau_balance_duree_moyenne_delais_de_paiements, expected_features_for_bureau_balance_loan_duration_categorised
    ]

validated_dfs_to_merge = []

for df_unchecked, expected_col_list in zip(dfs_to_merge, list_of_expected_features_per_df): #utiliser zip plutot que if and sinon ca va pas faire correspondre correctement les deux conditions par exemple ça va mettre df 2 avec liste de colonne 1
    if df_unchecked.empty :
        df_corrected = pd.DataFrame(columns=expected_col_list)
        validated_dfs_to_merge.append(df_corrected)
    else :
        validated_dfs_to_merge.append(df_unchecked)

merge_key = 'SK_ID_BUREAU'

merge_function = lambda left_df, right_df: pd.merge(left_df, right_df, on=merge_key, how='inner')
final_bureau_balance_features = reduce(merge_function, validated_dfs_to_merge)
#final_bureau_balance_features = final_bureau_balance_features.drop(columns=['0', 'C', 'X'])


bureau_for_credit_status = bureau.copy()
bureau_number_of_type_of_credits = bureau.groupby('SK_ID_CURR')['CREDIT_ACTIVE'].value_counts().unstack(fill_value=0).reset_index()
bureau_number_of_type_of_credits.columns.name = None

bureau_for_encoding_credit_types = bureau[['SK_ID_CURR', 'CREDIT_TYPE']]
bureau_credit_type_encoded = pd.get_dummies(
    bureau_for_encoding_credit_types, 
    columns=['CREDIT_TYPE'], 
    dtype=int
    )
bureau_liste_colonnes_encodees = bureau_credit_type_encoded.columns
bureau_liste_colonnes_a_agreger = [col for col in bureau_liste_colonnes_encodees if col != 'SK_ID_CURR']
bureau_credit_type_encoded_and_aggregated = bureau_credit_type_encoded.groupby('SK_ID_CURR')[bureau_liste_colonnes_a_agreger].sum().reset_index()

bureau_for_max_amount_overdue = bureau[['SK_ID_CURR', 'AMT_CREDIT_MAX_OVERDUE']]
bureau_for_max_amount_overdue.fillna(value=0, inplace=True)
bureau_for_max_amount_overdue = bureau_for_max_amount_overdue.groupby('SK_ID_CURR')['AMT_CREDIT_MAX_OVERDUE'].max().reset_index()

bureau_for_mean_amount_overdue_across_loans = bureau[['SK_ID_CURR', 'AMT_CREDIT_SUM_OVERDUE']]
bureau_for_mean_amount_overdue_across_loans = bureau_for_mean_amount_overdue_across_loans.groupby('SK_ID_CURR')['AMT_CREDIT_SUM_OVERDUE'].mean().reset_index()

bureau_for_credit_prolonged_count = bureau[['SK_ID_CURR', 'CNT_CREDIT_PROLONG']]
bureau_for_credit_prolonged_count = bureau_for_credit_prolonged_count.groupby('SK_ID_CURR')['CNT_CREDIT_PROLONG'].sum().reset_index()

bureau_for_proportions_repaid = bureau[['SK_ID_CURR', 'SK_ID_BUREAU', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM']]
bureau_for_proportions_repaid.dropna(subset='AMT_CREDIT_SUM', inplace=True)
bureau_for_proportions_repaid['AMT_CREDIT_SUM_DEBT_IS_MISSING'] = bureau_for_proportions_repaid['AMT_CREDIT_SUM_DEBT'].isnull().astype(int)
mask_active_missing_debt = bureau_for_proportions_repaid['AMT_CREDIT_SUM_DEBT'].isnull()
bureau_for_proportions_repaid.loc[mask_active_missing_debt, 'AMT_CREDIT_SUM_DEBT'] = bureau_for_proportions_repaid.loc[mask_active_missing_debt, 'AMT_CREDIT_SUM']
bureau_for_proportions_repaid_with_nan_values_treated = bureau_for_proportions_repaid.copy()
bureau_for_proportions_repaid_with_nan_values_treated = bureau_for_proportions_repaid_with_nan_values_treated.groupby('SK_ID_CURR')[['AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM']].sum().reset_index()
bureau_for_proportions_repaid_with_nan_values_treated['PROPORTION_REPAID'] = np.nan
# Rule 1: SUM = 0 and DEBT = 0  => Repaid = 0
mask_zero_both = (bureau_for_proportions_repaid_with_nan_values_treated['AMT_CREDIT_SUM'] == 0) & (bureau_for_proportions_repaid_with_nan_values_treated['AMT_CREDIT_SUM_DEBT'] == 0)
bureau_for_proportions_repaid_with_nan_values_treated.loc[mask_zero_both, 'PROPORTION_REPAID'] = 1.0
# Rule 2: DEBT < 0 => Repaid = 1
mask_neg_debt = bureau_for_proportions_repaid_with_nan_values_treated['AMT_CREDIT_SUM_DEBT'] < 0
bureau_for_proportions_repaid_with_nan_values_treated.loc[mask_neg_debt, 'PROPORTION_REPAID'] = 1.0
# Rule 4: DEBT>0 and SUM = 0
mask_pos_debt_null_sum = (bureau_for_proportions_repaid_with_nan_values_treated['AMT_CREDIT_SUM'] == 0) & (bureau_for_proportions_repaid_with_nan_values_treated['AMT_CREDIT_SUM_DEBT'] > 0)
bureau_for_proportions_repaid_with_nan_values_treated.loc[mask_pos_debt_null_sum, 'PROPORTION_REPAID'] = 0
# Rule 4: All other cases where SUM > 0
mask_calculate = (bureau_for_proportions_repaid_with_nan_values_treated['AMT_CREDIT_SUM'] > 0) & (bureau_for_proportions_repaid_with_nan_values_treated['PROPORTION_REPAID'].isnull()) # Only calculate where not set yet
bureau_for_proportions_repaid_with_nan_values_treated.loc[mask_calculate, 'PROPORTION_REPAID'] = 1.0 - (bureau_for_proportions_repaid_with_nan_values_treated.loc[mask_calculate, 'AMT_CREDIT_SUM_DEBT'] / bureau_for_proportions_repaid_with_nan_values_treated.loc[mask_calculate, 'AMT_CREDIT_SUM'])

bureau_for_deadline_in_the_past = bureau[['SK_ID_CURR', 'CREDIT_ACTIVE', 'DAYS_CREDIT_ENDDATE']].copy()
bureau_for_deadline_in_the_past['LOAN_ACTIVE_PAST_DEADLINE'] = np.where(
    (bureau_for_deadline_in_the_past['CREDIT_ACTIVE'] == 'Active') & (bureau_for_deadline_in_the_past['DAYS_CREDIT_ENDDATE'] < 0), # Condition
    1,                                             # Value if True
    0                                             # Value if False
)
bureau_for_deadline_in_the_past = bureau_for_deadline_in_the_past.groupby('SK_ID_CURR')['LOAN_ACTIVE_PAST_DEADLINE'].sum().reset_index()

bureau_for_mean_days_spent_overdue = bureau[['SK_ID_CURR', 'CREDIT_DAY_OVERDUE']]
bureau_for_mean_days_spent_overdue = bureau_for_mean_days_spent_overdue.groupby('SK_ID_CURR')['CREDIT_DAY_OVERDUE'].mean().reset_index()

dfs_to_merge = [bureau_number_of_type_of_credits,
bureau_credit_type_encoded_and_aggregated,
bureau_for_max_amount_overdue,
bureau_for_mean_amount_overdue_across_loans,
bureau_for_credit_prolonged_count,
bureau_for_proportions_repaid_with_nan_values_treated,
bureau_for_deadline_in_the_past,
bureau_for_mean_days_spent_overdue]

merge_key = 'SK_ID_CURR'

merge_function = lambda left_df, right_df: pd.merge(left_df, right_df, on=merge_key, how='inner')
final_bureau_features = reduce(merge_function, dfs_to_merge)

D:\Temp\TEMP\ipykernel_13580\3341190960.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bureau_for_max_amount_overdue.fillna(value=0, inplace=True)
D:\Temp\TEMP\ipykernel_13580\3341190960.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bureau_for_proportions_repaid.dropna(subset='AMT_CREDIT_SUM', inplace=True)
D:\Temp\TEMP\ipykernel_13580\3341190960.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [38]:
final_bureau_features.columns

Index(['SK_ID_CURR', 'Active', 'Bad debt', 'Closed', 'Sold',
       'CREDIT_TYPE_Another type of loan', 'CREDIT_TYPE_Car loan',
       'CREDIT_TYPE_Cash loan (non-earmarked)', 'CREDIT_TYPE_Consumer credit',
       'CREDIT_TYPE_Credit card', 'CREDIT_TYPE_Interbank credit',
       'CREDIT_TYPE_Loan for business development',
       'CREDIT_TYPE_Loan for purchase of shares (margin lending)',
       'CREDIT_TYPE_Loan for the purchase of equipment',
       'CREDIT_TYPE_Loan for working capital replenishment',
       'CREDIT_TYPE_Microloan', 'CREDIT_TYPE_Mobile operator loan',
       'CREDIT_TYPE_Mortgage', 'CREDIT_TYPE_Real estate loan',
       'CREDIT_TYPE_Unknown type of loan', 'AMT_CREDIT_MAX_OVERDUE',
       'AMT_CREDIT_SUM_OVERDUE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM_DEBT',
       'AMT_CREDIT_SUM', 'PROPORTION_REPAID', 'LOAN_ACTIVE_PAST_DEADLINE',
       'CREDIT_DAY_OVERDUE'],
      dtype='object')